In [3]:
from nba_api.stats.endpoints import leaguegamefinder
import pandas as pd
from nba_api.stats.endpoints import playbyplayv3
from nba_api.stats.endpoints import BoxScoreTraditionalV3
import time
import random
from requests.exceptions import ReadTimeout, ConnectionError, RequestException, Timeout
from json.decoder import JSONDecodeError
import re

def is_int_convertible(x):
    try:
        int(x)
        return True
    except (ValueError, TypeError):
        return False

def get_minutes_from_v3_clock(clock_str):
    if not isinstance(clock_str, str): return 0

    # Check for PTxxM format
    match = re.search(r'PT(\d+)M', clock_str)
    if match:
        return int(match.group(1))

    # Check for MM:SS format
    if ':' in clock_str:
        return int(clock_str.split(':')[0])

    return 0

# It will keep trying until it succeeds or you manually stop it.
def robust_fetch(endpoint_class, **kwargs):
    max_retries = 20  # High number, effectively infinite for short hiccups
    base_delay = 2    # Start waiting 2 seconds

    for attempt in range(max_retries):
        try:
            # Attempt the API call
            endpoint = endpoint_class(**kwargs)
            # Force a data access to ensure the request actually finished
            return endpoint.get_data_frames()[0]

        except (ReadTimeout, ConnectionError, Timeout, RequestException, JSONDecodeError) as e:
            # If any network/server error occurs:
            print(f"  [!] API Error on attempt {attempt+1}: {e}")
            print(f"  [i] Retrying in {base_delay} seconds...")

            time.sleep(base_delay)

            # Exponential Backoff: Wait longer next time (2s -> 4s -> 8s...)
            # Cap at 60 seconds to avoid waiting forever
            base_delay = min(base_delay * 2, 60)

        except Exception as e:
            # Catch unexpected Python errors (like syntax inside the API lib)
            print(f"  [!!!] Critical Unknown Error: {e}")
            break # Break so you don't infinite loop on a code bug

    return None # Return None if we completely failed after 20 tries

In [4]:
##.............data structures to be used.........##

#list of dictionary to append scraped data progressively
df_list=[] #every dict in this list represents a game

df= pd.DataFrame(columns=['GAME_ID',
                          'TEAM1',
                          'TEAM2', 'DATE', 'PF', 'CLUTCH', 'HIGHLIGHTS', 'PERFORMANCE'])

In [5]:
##.............Get data on Regular season games.........##
gamefinder = leaguegamefinder.LeagueGameFinder(season_nullable='2023-24',
                                              league_id_nullable='00',
                                              season_type_nullable='Regular Season')
games = gamefinder.get_data_frames()[0]
game_ids = games['GAME_ID'].unique().tolist() # get a list of distinct game_ids

In [39]:
print(f"Starting ingestion for {len(game_ids)} games...")

for i, game in enumerate(game_ids[1022:], start=1022):
    print(f"Processing Game {i + 1}/{len(game_ids)}: ID {game}")  # Progress indicator

    game_dict = {}

    try:
        game_dict['GAME_ID'] = game
        game_dict['TEAM1'] = str(games[games["GAME_ID"] == game].iloc[0]["TEAM_ABBREVIATION"])
        game_dict['TEAM2'] = str(games[games["GAME_ID"] == game].iloc[1]["TEAM_ABBREVIATION"])
        game_dict['DATE'] = str(games[games["GAME_ID"] == game].iloc[0]["GAME_DATE"])
        game_dict['PF'] = games[games["GAME_ID"] == game].iloc[0]["PF"] + games[games["GAME_ID"] == game].iloc[1]["PF"]

        #Fetch PlayByPlay
        pbp = robust_fetch(playbyplayv3.PlayByPlayV3, game_id=game)

        if pbp is None:
            print(f"  [x] Failed to fetch PBP for {game}. Skipping.")
            continue

        # --- A. Clutch Calculation ---
        period = pbp['period'].unique().max()
        if period == 5:
            game_dict['CLUTCH'] = 0.95
        elif period >= 6:
            game_dict['CLUTCH'] = 1.0
        else:
            #           Filter for 4th quarter +
            pbp_valid = pbp[pbp['period'] >= 4].copy()

            # Filter for last 5 minutes using helper (We apply the function to the 'clock' column)
            pbp_valid['MINUTES_REMAINING'] = pbp_valid['clock'].apply(get_minutes_from_v3_clock)
            pbp_valid = pbp_valid[pbp_valid['MINUTES_REMAINING'] < 5]

            # Calculate Margin Manually (Home - Away), Ensure we treat by forward fill if needed
            pbp_valid['scoreHome'] = pd.to_numeric(pbp_valid['scoreHome'], errors='coerce')
            pbp_valid['scoreAway'] = pd.to_numeric(pbp_valid['scoreAway'], errors='coerce')

            pbp_valid[['scoreHome', 'scoreAway']] = pbp_valid[['scoreHome', 'scoreAway']].ffill()
            pbp_valid = pbp_valid.dropna(subset=['scoreHome', 'scoreAway'])

            pbp_valid['margin'] = pbp_valid['scoreHome'] - pbp_valid['scoreAway']

            # Count possessions within +/- 5 points
            clutch_mask = (pbp_valid['margin'] <= 5) & (pbp_valid['margin'] >= -5)
            clutch_count = clutch_mask.sum()

            game_dict['CLUTCH'] = clutch_count

        # --- B. Performance Calculation ---
        stats_df = robust_fetch(BoxScoreTraditionalV3, game_id=game)

        if stats_df is None:
            print(f"  [x] Failed to fetch BoxScore for {game}. Skipping.")
            continue

        stats_df['GAME_SCORE'] = (
                stats_df['points'] + 0.4 * stats_df['fieldGoalsMade'] - 0.7 * stats_df['fieldGoalsAttempted'] -
                0.4 * (stats_df['freeThrowsAttempted'] - stats_df['freeThrowsMade']) + 0.7 * stats_df[
                    'reboundsOffensive'] +
                0.3 * stats_df['reboundsDefensive'] + stats_df['steals'] + 0.7 * stats_df['assists'] +
                0.7 * stats_df['blocks'] - 0.4 * stats_df['foulsPersonal'] - stats_df['turnovers']
        )

        performance_sum = 0.0
        for gs in stats_df['GAME_SCORE']:
            if gs >= 60.0:
                performance_sum += 0.95
            elif gs >= 50.0:
                performance_sum += 0.9
            elif gs >= 40.0:
                performance_sum += 0.8
            elif gs >= 30.0:
                performance_sum += 0.7

        game_dict['PERFORMANCE'] = performance_sum

        # --- C. Highlights Calculation ---
        pbp['description'] = pbp['description'].astype(str).str.upper()

        # Dunks: look for 'DUNK' in the single description column
        # Filter out 'MISS' to be safe, though usually 'Dunk' implies made in text
        if 'shotResult' in pbp.columns:
            dunks = pbp[
                (pbp['description'].str.contains("DUNK", case=False)) &
                (pbp['shotResult'] == 'Made')
                ].shape[0]
        else:
            # Fallback if specific column is missing
            dunks = pbp[
                (pbp['description'].str.contains("DUNK", case=False)) &
                (~pbp['description'].str.contains("MISS", case=False))
                ].shape[0]

        # Fix for threes: ensure we grab values safely
        threes = int(games[games["GAME_ID"] == game].iloc[0]["FG3M"] + games[games["GAME_ID"] == game].iloc[1]["FG3M"])

        blocks = int(games[games["GAME_ID"] == game].iloc[0]["BLK"] + games[games["GAME_ID"] == game].iloc[1]["BLK"])

        #calculate highlights/minutes
        total_highlights = dunks + (0.75 * threes) + blocks
        highlights_density = total_highlights / 48

        game_dict['HIGHLIGHTS'] = highlights_density

        df_list.append(game_dict)

        # Standard sleep
        time.sleep(random.uniform(0.5, 1.0))

    except Exception as e:
        print(f"  [!!!] Critical Error processing game {game}: {e}")
        # We continue to the next game even if this one crashes logic
        continue


Starting ingestion for 1230 games...
Processing Game 1023/1230: ID 0022300235
  [!] API Error on attempt 1: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)
  [i] Retrying in 2 seconds...
  [!] API Error on attempt 2: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)
  [i] Retrying in 4 seconds...
  [!] API Error on attempt 3: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)
  [i] Retrying in 8 seconds...
  [!] API Error on attempt 4: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)
  [i] Retrying in 16 seconds...
  [!] API Error on attempt 5: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
  [i] Retrying in 32 seconds...
  [!] API Error on attempt 6: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)
  [i] Retrying in 60 seconds...
Processing Game 1024/1230

In [43]:
dftemp = pd.DataFrame(df_list, columns=['GAME_ID','TEAM1','TEAM2', 'DATE', 'PF', 'CLUTCH', 'HIGHLIGHTS', 'PERFORMANCE'])
dftemp.to_csv("2023_Ei_metricsNotNormalized_beforeGEI.csv", index= False)

In [51]:
df_GEI23 = pd.read_csv(r"../GEI/2023_GEI_scraped.csv")
df_GEI24 = pd.read_csv(r"../GEI/2024_GEI_scraped.csv")
df_GEI25 = pd.read_csv(r"../GEI/2025_GEI_scraped.csv")
df_NN23 = pd.read_csv(r"2023_Ei_metricsNotNormalized_beforeGEI.csv", dtype={"GAME_ID": str})
df_NN24 = pd.read_csv(r"2024_Ei_metricsNotNormalized_beforeGEI.csv", dtype={"GAME_ID": str})
df_NN25 = pd.read_csv(r"2025_Ei_metricsNotNormalized_beforeGEI.csv", dtype={"GAME_ID": str})

In [53]:
#Adding a GEI column
def match_gei(row,df):
    matched = df[
        (df["Date"] == row["DATE"]) &
        (df["Score"].str.contains(row["TEAM1"], case=False, na=False)) &
        (df["Score"].str.contains(row["TEAM2"], case=False, na=False))
    ]
    return matched["Excitment"].mean() if not matched.empty else 0

df_NN23["GEI"] = df_NN23.apply(lambda row: match_gei(row, df_GEI23), axis=1) # 3 unmatched
df_NN24["GEI"] = df_NN24.apply(lambda row: match_gei(row, df_GEI24), axis=1) # 13 unmatched
df_NN25["GEI"] = df_NN25.apply(lambda row: match_gei(row, df_GEI25), axis=1) # 43 unmatched

In [66]:
#dropping missing data
df_NN25 = df_NN25[df_NN25["GEI"] != 0]
df_NN24 = df_NN24[df_NN24["GEI"] != 0]
df_NN23 = df_NN23[df_NN23["GEI"] != 0]


In [72]:
safe25= df_NN25.copy()
safe24= df_NN24.copy()
safe23= df_NN23.copy()

In [85]:
df_NN25= safe25
df_NN24= safe24
df_NN23= safe23

In [87]:
#Normalizing the data

# -------- 2025 --------
#HighLights
df_NN25["HIGHLIGHTS_NORM"] = (
        (df_NN25["HIGHLIGHTS"] - df_NN25["HIGHLIGHTS"].min()) /
        (df_NN25["HIGHLIGHTS"].max() - df_NN25["HIGHLIGHTS"].min())
    )
#Performance
df_NN25["PERFORMANCE_NORM"] = (
        (df_NN25["PERFORMANCE"] - df_NN25["PERFORMANCE"].min()) /
        (df_NN25["PERFORMANCE"].max() - df_NN25["PERFORMANCE"].min())
    )
#GEI
df_NN25["GEI_NORM"] = (
        (df_NN25["GEI"] - df_NN25["GEI"].min()) /
        (df_NN25["GEI"].max() - df_NN25["GEI"].min())
    )
#Clutch
df_OT25 = df_NN25[(df_NN25["CLUTCH"] == 0.95) | (df_NN25["CLUTCH"] == 1.0)].copy()
df_reg25 = df_NN25[(df_NN25["CLUTCH"] != 0.95) & (df_NN25["CLUTCH"] != 1.0)].copy()

df_reg25["CLUTCH_NORM"] = 0.9*(
        (df_reg25["CLUTCH"] - df_reg25["CLUTCH"].min()) /
        (df_reg25["CLUTCH"].max() - df_reg25["CLUTCH"].min())
    )
df_OT25["CLUTCH_NORM"]= df_OT25["CLUTCH"]
df_25 = pd.concat([df_reg25, df_OT25], axis=0).sort_index()
#PF
df_25["PF_NORM"]= (
        (df_25["PF"] - df_25["PF"].min()) /
        (df_25["PF"].max() - df_25["PF"].min())
    )

# -------- 2024 --------

#HighLights
df_NN24["HIGHLIGHTS_NORM"] = (
        (df_NN24["HIGHLIGHTS"] - df_NN24["HIGHLIGHTS"].min()) /
        (df_NN24["HIGHLIGHTS"].max() - df_NN24["HIGHLIGHTS"].min())
    )
#Performance
df_NN24["PERFORMANCE_NORM"] = (
        (df_NN24["PERFORMANCE"] - df_NN24["PERFORMANCE"].min()) /
        (df_NN24["PERFORMANCE"].max() - df_NN24["PERFORMANCE"].min())
    )
#GEI
df_NN24["GEI_NORM"] = (
        (df_NN24["GEI"] - df_NN24["GEI"].min()) /
        (df_NN24["GEI"].max() - df_NN24["GEI"].min())
    )
#Clutch
df_OT24 = df_NN24[(df_NN24["CLUTCH"] == 0.95) | (df_NN24["CLUTCH"] == 1.0)].copy()
df_reg24 = df_NN24[(df_NN24["CLUTCH"] != 0.95) & (df_NN24["CLUTCH"] != 1.0)].copy()

df_reg24["CLUTCH_NORM"] = 0.9*(
        (df_reg24["CLUTCH"] - df_reg24["CLUTCH"].min()) /
        (df_reg24["CLUTCH"].max() - df_reg24["CLUTCH"].min())
    )
df_OT24["CLUTCH_NORM"]= df_OT24["CLUTCH"]
df_24 = pd.concat([df_reg24, df_OT24], axis=0).sort_index()
#PF
df_24["PF_NORM"]= (
        (df_24["PF"] - df_24["PF"].min()) /
        (df_24["PF"].max() - df_24["PF"].min())
    )

# -------- 2023 --------

#HighLights
df_NN23["HIGHLIGHTS_NORM"] = (
        (df_NN23["HIGHLIGHTS"] - df_NN23["HIGHLIGHTS"].min()) /
        (df_NN23["HIGHLIGHTS"].max() - df_NN23["HIGHLIGHTS"].min())
    )
#Performance
df_NN23["PERFORMANCE_NORM"] = (
        (df_NN23["PERFORMANCE"] - df_NN23["PERFORMANCE"].min()) /
        (df_NN23["PERFORMANCE"].max() - df_NN23["PERFORMANCE"].min())
    )
#GEI
df_NN23["GEI_NORM"] = (
        (df_NN23["GEI"] - df_NN23["GEI"].min()) /
        (df_NN23["GEI"].max() - df_NN23["GEI"].min())
    )
#Clutch
df_OT23 = df_NN23[(df_NN23["CLUTCH"] == 0.95) | (df_NN23["CLUTCH"] == 1.0)].copy()
df_reg23 = df_NN23[(df_NN23["CLUTCH"] != 0.95) & (df_NN23["CLUTCH"] != 1.0)].copy()

df_reg23["CLUTCH_NORM"] = 0.9*(
        (df_reg23["CLUTCH"] - df_reg23["CLUTCH"].min()) /
        (df_reg23["CLUTCH"].max() - df_reg23["CLUTCH"].min())
    )
df_OT23["CLUTCH_NORM"]= df_OT23["CLUTCH"]
df_23 = pd.concat([df_reg23, df_OT23], axis=0).sort_index()
#PF
df_23["PF_NORM"]= (
        (df_23["PF"] - df_23["PF"].min()) /
        (df_23["PF"].max() - df_23["PF"].min())
    )


In [95]:
#CORRECTION OF INGESTION PIPELINE MISTAKE, BECAUSE GAMES WITH 1 CLUTCH POSSESSIONS ENDED UP WITH A 1.0 CLUTCH SCORE EVEN THOUGH IT IS RESERVED FOR DOUBLE OT GAMES
#DO THIS FOR 23,24 AND 25

# # filter all games in df with CLUTCH_NORM == 1.0
# games_high_clutch = df_23[df_23["CLUTCH_NORM"] == 1.0]["GAME_ID"].unique()
#
# # loop through each game_id
# for game in games_high_clutch:
#
#     try:
#         # fetch play-by-play data
#         pbp = playbyplayv3.PlayByPlayV3(game).get_data_frames()[0]
#
#         # get the maximum period in the play-by-play
#         max_period = pbp['period'].max()
#
#         # check if it is double OT (period >= 6)
#         if max_period < 6:
#             # change CLUTCH_NORM to 0.017143 for all matching rows
#             df_23.loc[
#                 (df_23["GAME_ID"] == game) &
#                 (df_23["CLUTCH_NORM"] == 1.0),
#                 "CLUTCH_NORM"
#             ] = 0.017143
#
#         # sleep to avoid rate limits
#         time.sleep(random.uniform(0.5, 1.0))
#
#     except Exception as e:
#         print(f"Error checking game {game}: {e}")


In [99]:
#exporting the target variable dataframes for training
df_25["GQS"]= 0.4*df_25["GEI_NORM"] + 0.25*df_25["CLUTCH_NORM"] + 0.2*df_25["PERFORMANCE_NORM"] + 0.15*df_25["HIGHLIGHTS_NORM"] - 0.05*df_25["PF_NORM"]

df_24["GQS"]= 0.4*df_24["GEI_NORM"] + 0.25*df_24["CLUTCH_NORM"] + 0.2*df_24["PERFORMANCE_NORM"] + 0.15*df_24["HIGHLIGHTS_NORM"] - 0.05*df_24["PF_NORM"]

df_23["GQS"]= 0.4*df_23["GEI_NORM"] + 0.25*df_23["CLUTCH_NORM"] + 0.2*df_23["PERFORMANCE_NORM"] + 0.15*df_23["HIGHLIGHTS_NORM"] - 0.05*df_23["PF_NORM"]

In [100]:
df_25

,GAME_ID,TEAM1,TEAM2,DATE,PF,CLUTCH,HIGHLIGHTS,PERFORMANCE,GEI,HIGHLIGHTS_NORM,PERFORMANCE_NORM,GEI_NORM,CLUTCH_NORM,PF_NORM,GQS
0,0022500326,SAS,ORL,2025-12-03,39,47.0,0.677083,0.0,8.2,0.268966,0.000000,0.485507,0.402857,0.38,0.316262
1,0022500325,IND,DEN,2025-12-03,45,0.0,0.739583,0.8,2.7,0.351724,0.380952,0.086957,0.000000,0.50,0.138732
2,0022500322,SAS,MEM,2025-12-02,50,40.0,0.776042,0.0,8.8,0.400000,0.000000,0.528986,0.342857,0.60,0.327308
3,0022500320,NYK,BOS,2025-12-02,35,19.0,0.833333,1.4,5.8,0.475862,0.666667,0.311594,0.162857,0.30,0.355065
4,0022500319,TOR,POR,2025-12-02,49,49.0,0.885417,0.0,6.9,0.544828,0.000000,0.391304,0.420000,0.58,0.314246
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
612,0022500327,ATL,LAC,2025-12-03,35,0.0,0.588542,0.0,3.5,0.151724,0.000000,0.144928,0.000000,0.30,0.065730
613,0022500330,SAC,HOU,2025-12-03,27,0.0,0.796875,0.0,5.0,0.427586,0.000000,0.253623,0.000000,0.14,0.158587
614,0022500332,DAL,MIA,2025-12-03,28,1.0,0.796875,0.0,5.7,0.427586,0.000000,0.304348,0.017143,0.16,0.182163
615,0022500329,CHI,BKN,2025-12-03,34,0.0,0.739583,0.0,4.9,0.351724,0.000000,0.246377,0.000000,0.28,0.137309
